In [ ]:
import re
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
sns.set(style='ticks', context='talk')
plt.rcParams['figure.figsize'] = (12, 6)

We'll use the same dataset of beer reviews.

In [ ]:
df = pd.read_csv('data/beer_subset.csv.gz', parse_dates=['time'], compression='gzip')
review_cols = ['review_appearance', 'review_aroma', 'review_overall',
               'review_palate', 'review_taste']
df.head()

A small teaser of the kind of stuff you can do

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
sns.countplot(hue='kind', y='stars', data=(df[review_cols]
                                           .stack()
                                           .rename_axis(['record', 'kind'])
                                           .rename('stars')
                                           .reset_index()),
              ax=ax, order=np.arange(0, 5.5, .5))
sns.despine()

# Groupby

Groupby is a fundamental operation to pandas and data analysis.

The components of a groupby operation are to

1. Split a table into groups
2. Apply a function to each group
3. Combine the results

In pandas the first step looks like

```python
df.groupby( grouper )
```

`grouper` can be many things

- Series (or string indicating a column in `df`)
- function (to be applied on the index)
- dict : groups by *values*
- `levels=[ names of levels in a MultiIndex ]`

In [ ]:
gr = df.groupby('beer_style')
gr

Haven't really done anything yet. Just some book-keeping to figure out which **keys** go with which **rows**. Keys are the things we've grouped by (each `beer_style` in this case).

The last two steps, apply and combine, are just:

In [ ]:
gr.agg('mean')

This says apply the `mean` function to each column. Non-numeric columns (nusiance columns) are excluded. We can also select a subset of columns to perform the aggregation on.

In [ ]:
gr[review_cols].agg('mean')

`.` attribute lookup works as well.

In [ ]:
gr.abv.agg('mean')

Certain operations are attached directly to the `GroupBy` object, letting you bypass the `.agg` part

In [ ]:
gr.abv.mean()

<div class="alert alert-success">
    <b>Exercise</b> Find the `beer_style`s with the greatest variance in `abv`.
</div>

- hint: `.std` calculates the standard deviation, and is available on `GroupBy` objects like `gr.abv`.
- hint: use `.sort_values` to sort a Series by the values (it took us a while to come up with that name)

In [ ]:
# your code goes here

In [ ]:
%load -r 15:17 solutions/solutions_groupby.py

Now we'll run the gamut on a bunch of grouper / apply combinations.
Keep sight of the target though: split, apply, combine.

- Grouper: Controls the output index
    * single grouper -> `Index`
    * array-like grouper -> `MultiIndex`
- Subject (Groupee): Controls the output data values
    * single column -> `Series` (or DataFrame if multiple aggregations)
    * multiple columns -> `DataFrame`
- Aggregation: Controls the output columns
    * single aggfunc -> `Index` in the colums
    * multiple aggfuncs -> `MultiIndex` in the columns (Or 1-D Index groupee is 1-d)

Multiple Aggregations on one column

In [ ]:
gr['review_aroma'].agg(['mean', np.std, 'count']).head()

Single Aggregation on multiple columns

In [ ]:
gr[review_cols].mean()

Multiple aggregations on multiple columns

In [ ]:
gr[review_cols].agg(['mean', 'count', 'std'])

Hierarchical Indexes in the columns can be awkward to work with, so I'll usually
move a level to the Index with `.stack`.

In [ ]:
multi = gr[review_cols].agg(['mean', 'count', 'std']).stack(level=0)
multi.head(10)

You can group by **levels** of a MultiIndex.

In [ ]:
multi.groupby(level='beer_style')['mean'].agg(['min', 'max'])

Group by **multiple** columns

In [ ]:
df.groupby(['brewer_id', 'beer_style']).review_overall.mean()

In [ ]:
df.groupby(['brewer_id', 'beer_style'])[review_cols].mean()

<div class="alert alert-success">
    <b>Exercise</b>: Plot the relationship between review length (the `text` column) and average `review_overall`.
</div>

Hint: Break the problem into pieces:

- Find the **len**gth of each reivew (remember the `df.text.str` namespace?)
- Group by that Series of review lengths
- Find the `mean` `review_overall` by review length
- I used `style='k.'` in the plot 

In [ ]:
# Your solution

In [ ]:
%load -r 1:5 solutions/solutions_groupby.py

<div class="alert alert-info">
  <b>Bonus exercise</b>
</div>

- Try grouping by the number of words.
- Try grouping by the number of sentances.

Remember that `str.count` accepts a regular expression.

Don't worry too much about these, especially if you don't remember the syntax
for regular expressions (I never can). Just jump to the next exercise.

In [ ]:
# Your code goes here

In [ ]:
%load -r 18:20 solutions/solutions_groupby.py

<div class="alert alert-success">
  <b>Exercise</b>: Which **brewer** (`brewer_id`) has the largest gap between the min and max `review_overall` for two of their beers.
</div>

Hint: You'll need to do this in two steps.

1. Find the average `review_overall` by `brewer_id` and `beer_name`.
2. Find the difference between the max and min by brewer (rembember `.groupby(level=)`)

In [ ]:
# Your code goes here. You've got this!

In [ ]:
%load -r 6:13 solutions/solutions_groupby.py

Create our own "kind" of beer, which aggregates `style`.

In [ ]:
style = df.beer_style.str.lower()
style.head()

In [ ]:
kinds = ['ipa', 'apa', 'amber ale', 'rye', 'scotch', 'stout', 'barleywine', 'porter', 'brown ale', 'lager', 'pilsner',
         'tripel', 'biter', 'farmhouse', 'malt liquour', 'rice']

In [ ]:
expr = '|'.join(['(?P<{name}>{pat})'.format(pat=kind, name=kind.replace(' ', '_')) for kind in kinds])
expr

In [ ]:
beer_kind = (style.replace({'india pale ale': 'ipa',
                            'american pale ale': 'apa'})
            .str.extract(expr, expand=False).fillna('').sum(1)
            .str.lower().replace('', 'other'))
beer_kind.head()

In [ ]:
df.groupby(['brewer_id', beer_kind]).review_overall.mean()

In [ ]:
df.groupby(['brewer_id', beer_kind]).beer_id.nunique().unstack(1).fillna(0)

We've seen a lot of permutations among number of groupers, number of columns to aggregate, and number of aggregators.
In fact, the `.agg`, which returns one row per group, is just one kind of way to combine the results. The three ways are

- `agg`: one row per results
- `transform`: identicaly shaped output as input
- `apply`: anything goes


# Transform

Combined Series / DataFrame is the same shape as the input. For example, say you want to standardize the reviews by subtracting the mean.

In [ ]:
def de_mean(reviews):
    s = reviews - reviews.mean()
    return s

In [ ]:
de_mean(df.review_overall)

We can do this at the *person* level with `groupby` and `transform`.

In [ ]:
df['review_overall_demeaned'] = df.groupby('profile_name').review_overall.transform(de_mean)

In [ ]:
df[['review_overall', 'review_overall_demeaned']].head()

# Apply

So there's `gr.agg`. and `gr.transform`, and finally `gr.apply`. We're going to skip apply for now. I have an example in a later notebook. 

# Resample

Resample is a special kind of groupby operation for when you have a `DatetimeIndex`.

In [ ]:
%matplotlib inline

In [ ]:
flights = pd.read_csv("data/flights.csv.gz",
                      parse_dates=['fl_date', 'crs_dep_time', 'dep_time',
                                   'crs_arr_time', 'arr_time'])
flights.head()

In [ ]:
dep = flights.crs_dep_time.value_counts().sort_index()
dep.head()

In [ ]:
dep.plot();

In [ ]:
resampler = dep.resample('H')
resampler.mean().plot()

In [ ]:
dep.resample('3H').mean().plot()

In [ ]:
dep.resample("D").sum().plot()

In [ ]:
daily = dep.resample("D").sum()
daily

In [ ]:
daily.rolling(7).mean().plot()

In [ ]:
flights.head()

In [ ]:
flights['dep_delay'] = (flights['dep_time'] - flights['crs_dep_time']).dt.seconds / 60

In [ ]:
delay = flights.set_index('crs_dep_time').dep_delay.sort_index()
delay.resample('H').agg(['mean', 'sum', 'count']).rolling(8)['mean'].mean().plot()

<div class="alert alert-info">
  <b>Extra Exercises</b>
</div>

Which carriers have the longest delays, on average?

Is there a seasonal component to the trend? (Hint: look at statsmodels' `sm.tsa.seasonal_decompose`)

How do delays vary by time of day (say hour)? By day of week? (Hint: datetime columns have a `.dt` namespace, similar to how string columns have a `.str` namespace; this provides access to all the components of the timestamp).

What's the relationship between distance flown and the *arrival* delay?